<a href="https://colab.research.google.com/github/jae-gyeong/marchine_learning_start/blob/master/19_10_08GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01 데이터 가져오기

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


## 02 기본 옵션 설정

In [0]:
total_epoch = 100 # epoch 수 설정
batch_size = 100 # 배치 사이즈
learning_rate = 0.0002 # 학습률
# 신경망 레이어 구성 옵션
n_hidden = 256 # 은닉층 노드
n_input = 28 * 28 # 입력
n_noise = 128 # 생성기의 입력값으로 사용할 노이즈의 크기

##02 신경망 모델 구성
- 노이즈를 이용하여 데이터 생성
- 비지도학습이므로 Y가 없음

In [0]:
# GAN 도 Unsupervised 학습이므로 Autoencoder 처럼 Y 를 사용하지 않습니다.
X = tf.placeholder(tf.float32, [None, n_input])

# 생성망에 들어갈 입력 데이터 (정규분포를 따르는 128개의 숫자 데이터), 입력값
Z = tf.placeholder(tf.float32, [None, n_noise])

## 생성자 신경망 변수 선언
-n_noise : 128, n_hidden : 256
- 첫번째 가중치와 편향은 은닉층으로 출력하기 위한 변수
- 두 번째 가중치와 편향은 출력층에서 사용할 변수
- 두 번째 가중치의 변수 크기는 실제 이미지의 크기와 같아야 함.

In [0]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))


## 2-1 생성자(G) 신경망 구성
- 무작위 생성한 노이즈를 받아, 가중치와 편향을 반영하여 은닉층 구성.
- sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [0]:
# (판별망) 입력층 -> 은닉층으로( 판별기가 신경망에 사용하는 변수들입니다.)
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))


# 은닉층 -> 출력층( 판별기의 최종 겨로가값은 얼마나 진짜와 가깝냐를 판단하는 한개의 스칼라값입니다.)
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))


## 2-1 생성자(G) 신경망 구성
##  노이즈 데이터 발생을 위한 노이즈 생성함수
- deg get_noise(데이터 개수, 노이즈 개수)


- 무작위 생성한 노이즈를 받아, 가중치와 편향을 반영하여 은닉층 구성.
- sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [0]:
def get_noise(batch_size, n_noise):
  return np.random_normal(size=(batch_size, n_noise))

In [0]:
def generator(noise_z):
 hidden = tf.nn.relu( tf.matmul(noise_z, G_W1) + G_b1)
 output = tf.nn.sigmoid( tf.matmul(hidden, G_W2) + G_b2)
 return output

## 2-2 구분자(D) 신경망 구성
- 구분자 신경망 구성, 가중치와 편향을 반영한 데이터 출력
- sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [0]:
def discriminator(inputs):
 hidden = tf.nn.relu( tf.matmul(inputs, D_W1) + D_b1)
 output = tf.nn.sigmoid( tf.matmul(hidden, D_W2) + D_b2)
 return output


In [0]:
# 노이즈를 이용해 랜덤한 이미지를 생성합니다.
# Z에는 실행 시, noise가 입력됨.
G = generator(Z)
# 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다.
D_fake = discriminator(G)
# 진짜 이미지를 이용해 판별한 값을 구합니다.
D_real = discriminator(X)

- 경찰을 학습(구분자가 가짜라고 판단하도록 하는 손실값)시킬 때,
  - D_real은 1에 가까워야 함.
  - 가짜 이미지 판별값(D_fake)은 0에 가까워야 함.


## GAN의 모델의 최적화
- loss_G(생성자)와 loss_D(구분자)를 최대화 하는 것.
  - 단, 서로의 손실이 연관되어 있어, 두 손실값이 같이 증가가 어려움.
- loss_D를 최대화하기 위해서는 D_fake값을 최소화시킴.
- 판별기에 진짜 이미지를 넣었을 때에도 최대값을 : tf.log(D_real)
- 가짜 이미지를 넣었을 때에도 최대값을 : tf.log(1 - D_fake)

In [0]:
# 경찰관의 성능 -> 가능한 높은 값
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_fake))


- D_fake(가짜 이미지 판별값)은 1에 가깝게 만들면 된다.(위조 지폐범의 학습)
- 위조 지폐범의 학습은 D_fake(가짜)값을 최대화 한다.
- 가짜 이미지를 넣었을 때, 판별기가 최대한 실제 이미지라고 판단하도록 생성기 신경망을 학습

In [0]:
# 위조지폐범의 성능 -> 가능한 높은 값
# 결국 D_fake 값을 최대화하는 것이므로 다음과 같이 사용할 수 있습니다.)
loss_G = tf.reduce_mean(tf.log(D_fake))

In [0]:
# loss_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# loss_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용하여 최적화를 합니다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]


**즉 GAN의 학습은 loss_D와 loss_G 를 최대화 하는것.**

In [0]:
# GAN 논문의 수식에 따르면 loss 를 극대화 해야하지만,
# minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

## 2-3 생성자의 입력인 노이즈 발생을 위한 노이즈 생성 함수

In [0]:
# 랜덤한 노이즈(Z)를 만듭니다.
def get_noise(batch_size, n_noise):
 return np.random.normal(size=(batch_size, n_noise))

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [68]:
for epoch in range(20):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size) # 판별망의 데이터 
    noise = get_noise(batch_size, n_noise) # 생성망에 들어가는 입력
    
    _, loss_val_D = sess.run([train_D, loss_D], 
                             feed_dict={X:batch_xs, Z:noise})
    _, loss_val_G = sess.run([train_G, loss_G],
                             feed_dict={Z:noise})
    

  if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise) # 노이즈 생성.
    samples = sess.run(G, feed_dict={Z: noise})
    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
    
    for i in range(sample_size):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i], (28, 28)))
      
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)

  print("Epoch : {}, 판별망의 성능 : {:.4f}, 생성망의 성능 : {:.4f}".format(epoch, 
                                                           loss_val_D,
                                                           loss_val_G))
    
print("최적화 완료!")

Epoch : 0, 판별망의 성능 : -0.1946, 생성망의 성능 : -2.9764
Epoch : 1, 판별망의 성능 : -0.2604, 생성망의 성능 : -2.9937
Epoch : 2, 판별망의 성능 : -0.2559, 생성망의 성능 : -2.9332
Epoch : 3, 판별망의 성능 : -0.2861, 생성망의 성능 : -3.3403
Epoch : 4, 판별망의 성능 : -0.1715, 생성망의 성능 : -3.0916
Epoch : 5, 판별망의 성능 : -0.2621, 생성망의 성능 : -2.7731
Epoch : 6, 판별망의 성능 : -0.2703, 생성망의 성능 : -2.9081
Epoch : 7, 판별망의 성능 : -0.3335, 생성망의 성능 : -2.8540
Epoch : 8, 판별망의 성능 : -0.3475, 생성망의 성능 : -2.8796
Epoch : 9, 판별망의 성능 : -0.4739, 생성망의 성능 : -2.7442
Epoch : 10, 판별망의 성능 : -0.5065, 생성망의 성능 : -2.1829
Epoch : 11, 판별망의 성능 : -0.4016, 생성망의 성능 : -2.5612
Epoch : 12, 판별망의 성능 : -0.3536, 생성망의 성능 : -2.5251
Epoch : 13, 판별망의 성능 : -0.3894, 생성망의 성능 : -2.5349
Epoch : 14, 판별망의 성능 : -0.3572, 생성망의 성능 : -2.7632
Epoch : 15, 판별망의 성능 : -0.3027, 생성망의 성능 : -2.6875
Epoch : 16, 판별망의 성능 : -0.5081, 생성망의 성능 : -2.7252
Epoch : 17, 판별망의 성능 : -0.6226, 생성망의 성능 : -2.6360
Epoch : 18, 판별망의 성능 : -0.5073, 생성망의 성능 : -2.2959
Epoch : 19, 판별망의 성능 : -0.4687, 생성망의 성능 : -2.5997
최적화 완료!
